### **Moondream2**

In [2]:
!pip install -U bitsandbytes
!pip install transformers einops
!pip install flash-attn --no-build-isolation
!pip install tqdm
!pip install timm
!pip install decord

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 37.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.6.3-cp310-cp310-linux_x86_64.whl size=187309225 sha256=237ef9c6157db394e1ddde4ba609a21ebb98382377a27041edc09318801a6f24
  Stored in directory: /root/.cache/pip/wheels/7e/e3/c3/89c7a2f3c4adc07cd1c675f8bb7b9ad4d18f64a72bccdfe826
Successfully built flash-attn
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 64.9 MB/s eta 0:00:00


In [3]:
import torch
import random
import requests
import pandas as pd
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from tqdm import tqdm

In [4]:
model_id = "Meghnad/moondream2-dimextract-ft-464-10"
revision = "2024-08-26"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    # revision=revision,
    trust_remote_code=True,
    torch_dtype=torch.float16,
).eval().to("cuda")
tokenizer = AutoTokenizer.from_pretrained('vikhyatk/moondream2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/403 [00:00<?, ?B/s]

configuration_moondream.py:   0%|          | 0.00/3.34k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vikhyatk/moondream2:
- configuration_moondream.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


moondream.py:   0%|          | 0.00/7.20k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [5]:
# trnhdf = pd.read_csv("/content/train.csv")
df = pd.read_csv("/content/height_4.csv")


In [ ]:
for index, row in tqdm(df.iterrows()):

    url = row["image_link"]
    response = requests.get(url)
    image = Image.open(BytesIO(response.content))
    enc_image = model.encode_image(image).to('cuda')
    predicted_value = model.answer_question(enc_image, "what is the height mentioned in the image?", tokenizer)
    df.at[index, 'prediction'] = predicted_value
    df.to_csv("moon_2_height_4_predictions.csv", index=False)

1418it [13:03,  1.93it/s]